In [3]:
import ee
import folium
import geopandas as gpd
# allow images to display in the notebook
from IPython.display import Image
import pandas as pd

In [4]:
ee.Authenticate()
ee.Initialize(project="campfire-mapping")

In [5]:
# coordinates of the Camp Fire
lat =  39.444012
lon = -121.833619

# point of interest as an ee.Geometry
poi = ee.Geometry.Point(lon,lat)

# start date of range to filter for
start_date = '2018-10-01'

# end date
end_date = '2019-01-31'

# get the satellite data
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
            .filterBounds(poi)\
            .filterDate(start_date,end_date)

In [ ]:
# how many images did we get
image_count = landsat.size().getInfo()

# when was this image taken
landsat.first().get('DATE_ACQUIRED').getInfo()

# what bands did we get
print(landsat.first().bandNames().getInfo())

0.05
['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10', 'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA', 'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT']


In [ ]:
landsat_list = landsat.toList(landsat.size())

parameters = {
    'min': 7000,
    'max': 16000,
    'dimensions': 800, # square size in pixels
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'] # bands to display (r,g,b)
}

data = []
# loop through each image and display it
for i in range(image_count):

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    
    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    # print the image info
    print(f"Image #{i} {date} -- Cloud cover: {cloud}")
    
    # display the image
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

    # data to list
    this_data = [i, date, cloud]

    # append the data 
    data.append(this_data)
    

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['Image #', 'Date', 'Cloud Cover'])

Image # 0 2018-10-07 Cloud cover: 0.05


Image # 1 2018-10-23 Cloud cover: 73.04


Image # 2 2018-11-08 Cloud cover: 11.83


Image # 3 2018-11-24 Cloud cover: 67.16


Image # 4 2018-12-10 Cloud cover: 56.09


Image # 5 2018-12-26 Cloud cover: 5.99


Image # 6 2019-01-11 Cloud cover: 80.06


Image # 7 2019-01-27 Cloud cover: 5.21


In [8]:
print(df)

   Image #        Date  Cloud Cover
0        0  2018-10-07         0.05
1        1  2018-10-23        73.04
2        2  2018-11-08        11.83
3        3  2018-11-24        67.16
4        4  2018-12-10        56.09
5        5  2018-12-26         5.99
6        6  2019-01-11        80.06
7        7  2019-01-27         5.21


In [11]:
# find the index of only those images whose cloud cover is below 15% (usable range)
usable_image_indexes = df.loc[df["Cloud Cover"] < 15].index.tolist()

# Define a region of interest with a buffer zone of 20 km
roi = poi.buffer(20000) # meters

parameters = {
    'min': 6000,
    'max': 16000,
    'dimensions': 800,
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'region':roi
}

for i in usable_image_indexes:
    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    print('Image #',i,date,'Cloud cover:',cloud)
    
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

Image # 0 2018-10-07 Cloud cover: 0.05


Image # 2 2018-11-08 Cloud cover: 11.83


Image # 5 2018-12-26 Cloud cover: 5.99


Image # 7 2019-01-27 Cloud cover: 5.21


In [12]:
# ndvi palette: red is low, green is high vegetation
palette = ['red', 'yellow', 'green']

# anything below 0.2 will be red, 0.2-0.4 is yellow, and > 0.4 is green
ndvi_parameters = {
    'min': 0,
    'max': 0.4,
    'dimensions': 512,
    'palette': palette,
    'region': roi
}

for i in usable_image_indexes:
    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    
    # print some information
    print('Image #', i, date)
    
    # display the image
    display(Image(
        url = ee.Image(landsat_list.get(i)).\
        normalizedDifference(['SR_B5', 'SR_B4']).\
        getThumbUrl(ndvi_parameters)
    ))

Image # 0 2018-10-07


Image # 2 2018-11-08


Image # 5 2018-12-26


Image # 7 2019-01-27


In [15]:
# Google function that allows ee layers on folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

In [16]:
my_map = folium.Map(location=[lat, lon], zoom_start=10)

# Add a layer for each satellite image of interest (before, during and after)
for i in usable_image_indexes:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    my_map.add_ee_layer(
        ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']), 
        ndvi_parameters, 
        name=date
    )
    
# Add a layer control panel to the map
folium.LayerControl(collapsed = False).add_to(my_map)

# Display the map.
display(my_map)

In [27]:
my_map.save("campfire_map.html")